In [1]:
import pandas as pd
import bs4
from selenium import webdriver
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as cond
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException

In [4]:
# ## Getting links dataframe with only column of links.
# # links_df=pd.read_excel(r'/home/sumit/Scrape_Project/Cities_and_Links.xlsx')['Links']
# links1=pd.read_csv(r'/home/sumit/Scrape_Project/Scrapped_Links.csv').reset_index(drop=True)
links=pd.read_csv(r'D:\Personal\New folder\Links\Bangalore\Bangalore.csv').reset_index(drop=True)

In [5]:
links_df=links.drop_duplicates(subset='Links').reset_index(drop=True)['Links']
# links_df1=links1.drop_duplicates(subset='Links').reset_index(drop=True)['Links']

In [6]:
# links_df=links_df.append(links_df1,ignore_index=True)

In [7]:
links_df.drop_duplicates(inplace=True)

In [8]:
# links_df=links_df.drop(index=links_df.loc[links_df.isin(links_df1)].index).reset_index(drop=True)

In [9]:
# Details_df=pd.DataFrame(columns=["Name_Institution","Contact_No","Email","Contact_Person","Location","Address"])

In [10]:
def find_details(contact_texts):
    '''Function to get the every detail from Contact Details Section of website'''
    mob=[]
    for strings in contact_texts[1:]:
        if re.match(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",strings):
            mail=strings
        elif re.match(r"^(\d{10}|\d{12}|\d{9})",strings):
            mob.append(strings)
#             print(mob)
        elif re.match(r"^(/..)|(\+\d+)",strings):
            mob.append(strings)
#             print(mob)
        elif re.match(r"^(?:(?:\+|0{0,2})91(\s*[\-]\s*)?|[0]?)?[789]\d{9}$",strings):
            mob.append(strings)
        elif re.match(r"www.\w+",strings):
            web=strings
        elif re.match(r'http\w*',strings):
            web=strings
    if 'web' not in locals():
        web=None
    if 'mail' not in locals():
        mail=None
    mob="".join(mob)
    return mob,mail,web

In [11]:
Details_df=pd.DataFrame(columns=["Name_Institution","Contact_Person","Contact_No","Email","Location","Website","Address"])

In [16]:
driver= webdriver.Chrome(r'C:\Users\Sumit Anand\Downloads\Programs\chromedriver_win32\chromedriver.exe')##Defining the driver.
driver.implicitly_wait(10)## Implicit wait to load browser.
#     Details_df=pd.DataFrame(columns=["Name_Institution","Contact_Person","Contact_No","Email","Location","Website","Address"])
for x in range(0,0+len(links_df)):
    if x%200==0:
        driver.delete_all_cookies()
    try:
        driver.get(links_df[x])
        WebDriverWait(driver,10).until(cond.presence_of_all_elements_located((By.CLASS_NAME,'r_source-review')))## Implicit wait to load webpage.
    except TimeoutException:
        continue
#         driver.get(links_df[x])
#         WebDriverWait(driver,10).until(cond.presence_of_all_elements_located((By.CLASS_NAME,'r_source-review')))## Implicit wait to load webpage.
    except WebDriverException:
        driver= webdriver.Chrome()##Defining the driver.
        driver.implicitly_wait(10)
        driver.get(links_df[x])
        WebDriverWait(driver,10).until(cond.presence_of_all_elements_located((By.CLASS_NAME,'r_source-review')))
    except: pass## Implicit wait to load webpage.
            #         time.sleep(10)## 10 Seconds wait for all elements to be loaded.
    details_dict={}## Dictionary for each detail to append into dataframe.
    col_name=list(Details_df.columns)
        ## Name of Institution
    details_dict[col_name[0]]=driver.find_element_by_xpath("/html/body/div[7]/div[2]/div[5]/div[1]/h1").text
        ## Location
    details_dict[col_name[4]]=driver.find_element_by_xpath("/html/body/div[7]/div[2]/div[7]/div[3]/div[3]/div[2]").text
        ## Address
    add=driver.find_element_by_id('contact').text
#         print(add)
    details_dict[col_name[6]]=add.replace("\n",",")
#         ",".join(add.split("\n")[1:-3])
        ## Contact Details block from website.
    contact=driver.find_elements_by_class_name('r_source-review')[-2]
    if len(contact.text.split("\n"))>1:
        if not re.match(r"\d+",contact.text.split("\n")[1]):
                ##Contact Person
            details_dict[col_name[1]]=contact.text.split("\n")[1]
#             print(details_dict[col_name[1]])
        else: details_dict[col_name[1]]=None        
        
        mob,mail,web=find_details(contact.text.split("\n"))
            ## Phone No
        details_dict[col_name[2]]=mob
            ## Mail
        details_dict[col_name[3]]=mail
            ## Website
        details_dict[col_name[5]]=web
    else:
             ## Phone No
        details_dict[col_name[2]]=None
            ## Mail
        details_dict[col_name[3]]=None
            ## Website
        details_dict[col_name[5]]=None
#         phno=contact_p.text.split("\n")[2]
#         if len(contact_p.text.split("\n"))>3:
#             if re.match(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",contact_p.text.split("\n")[3]):
#                 email=contact_p.text.split("\n")[3]
#             else:
#                 phno=list(phno).append(contact_p.text.split("\n")[3])
#         if len(contact_p.text.split("\n"))>3:
#         websit=contact_p.text.split("\n")[3]
    Details_df=Details_df.append(details_dict,ignore_index=True)
    print(x,details_dict[col_name[0]])
#         break
driver.close()

0 CHILDHOOD CUDDLE AND CARE
1 JYESHTA PLAY HOME AND DAY CARE
2 STEPUP CHILD CARE AND ACTIVITY CENTRE
3 KIDS CASTLE PRESCHOOL
4 KARA 4 KIDS
5 ELEMENTS KIDS
6 MUDDIEE TOES
7 KIDS SPACE ACADEMY
8 THE NEST DAY CARE CENTER
9 BUDDING SAPLING
10 IMPERIAL MOTHERCARE
11 GEETHANJALI MONTESSORI
12 APPLE KIDS
13 ANUBHAV PLAYHOUSE
14 HELLO KIDS
15 SERRA INTERNATIONAL
16 LITTLE EINSTEINS
17 KEPHEE PRE SCHOOL
18 GURUKUL KIDS
19 MALIS CRECHE AND PLAY HOME CENTRE
20 FEATHERTOUCH INTERNATIONAL
21 OI PLAY SCHOOL
22 SCRIBBLES PLAYSCHOOL
23 PETALS MONTESSORI SCHOOL
24 MATHRU DAY CARE CENTRE
25 BODHI KIDDS HOODI
26 SHEMROCK ARIYAN
27 TODDLERS TOWN
28 SMART KIDS CENTRE FOR EXCELLENCE
29 TINYFEET DAY CARE SERVICE
30 KIDS CASTLE PRESCHOOL
31 SATTVA PRESCHOOL
32 LITTLE CHAMPS PRESCHOOL
33 HELLO KIDS
34 KIDZEE
35 LITTLE ELLY PRESCHOOL
36 LITTLE MILLENNIUM
37 HELLO KIDS
38 LITTLE MILLENNIUM
39 TOTS HOUSE PLAY HOUSE & DAY CARE CENTRE
40 LEO KIDS
41 PODAR JUMBO KIDS PLUS
42 BLOSSOM BUDS
43 LITTLE GEMZ EDUCATION
44 

351 MAPLEBEAR CANADIAN PRESCHOOL
352 WINNIE KIDS
353 SPECTRUM KIDS
354 BLOOMING KIDS
355 BINDU DAY CARE
356 HELLO KIDS
357 LITTLE ELLY PRESCHOOL
358 LITTLE MILLENNIUM
359 BUMBLE BEE MONTESSORI SCHOOL
360 LITTLE ELLY PRESCHOOL
361 TRUSHNA DAY CARE
362 ANGELA ROYES CHILD DEVELOPMENT CENTER PRESCHOOL & DAYCARE
363 HELLO KIDS
364 PRAGYAA PRE SCHOOL & DAYCARE
365 GREEN LAWNS SCHOOL
366 MAPLEBEAR CANADIAN PRESCHOOL
367 OI PLAY SCHOOL
368 MASTERS PRAGNAVAHINI
369 BACHPAN PLAY SCHOOL
370 BRIGHT KID MONTESSORI HOUSE
371 HELLO KIDS
372 HELLO KIDS
373 LITTLE ELLY PRESCHOOL
374 KIDZ PATHSHALA
375 LITTLE ELLY PRESCHOOL
376 LITTLE KINGDOM
377 TOM & JERRY CRECHE
378 MAPLEBEAR CANADIAN PRESCHOOL
379 OM VIVEKANANDA PLAY HOME
380 KID ZONE
381 BLOSSOMS INTERNATIONAL PLAY SCHOOL
382 SUMMER HILL PLAY HOME
383 TARA EARLY LEARNING CENTRE
384 KEYKIDZ
385 KIDZ STEPPING STONE
386 ESPERANZA
387 PODDAR HAPPY KIDS
388 NATANIA BABY SCHOOL
389 PLANET BLOOMINGDALE
390 SERRA INTERNATIONAL
391 PODAR HAPPY KIDS
392 DAY 

688 JOY BERRY
689 LITTLE LEGENDS NURSERY AND DAY CARE
690 SUNRISE KIDS PLAY HOME & ACTIVITY CENTER
691 BRAINWORKS
692 COLORS PRESCHOOL & DAYCARE
694 AMERICAN KIDZ
695 KIDS TIME PRESCHOOL
696 OI PLAYSCHOOL
697 SHINEY PEARL DAY CARE
698 SHISHU CARE
699 SPECIALME DAYCARE
700 SUNNYSIDE DAYCARE & PLAYHOME
701 HOPSTART JUNIORS
702 YOUR KUDOS
703 JANANI CHILD DAY CARE CENTER
704 KIDS CASTLE PRESCHOOL
705 OI PLAYSCHOOL
706 SMART KIDS DAY CARE & PLAYGROUP
707 SMILEYS DAYCARE
708 KIDZEE
709 KLAY PREP SCHOOL & DAYCARE
710 KLAY PREP SCHOOL & DAYCARE
711 RED BRIDGE INTERNATIONAL SCHOOL
712 TODDLERS KINGDOM
713 LITTLE MILLENNIUM
714 MORNING STAR MONTESSORI & PRE-SCHOOL
715 KIDS CASTLE PRESCHOOL
716 KIDS CASTLE PRESCHOOL
717 SESAME STREET PRESCHOOL
718 DRS EDIFY KIDS
719 KIDS PRISM
720 ITS MY WORLD CHILD CARE CENTER & PLAY SCHOOL
721 KADBURY KIDS
722 OI PLAYSCHOOL
723 KLAY PREP SCHOOL & DAYCARE
724 MY FIRST STEP
725 KIDDIE GALAXY PRESCHOOL
726 LITTLE BEES INTERNATIONAL PRE SCHOOL
727 MORNING STAR DAY

1028 MUDDIEE TOES
1029 LITTLE EINSTEINS
1030 VISHNU DAY CARE
1031 VIBGYOR KIDS
1032 BRIGHT MINDS
1033 I PLAY I LEARN
1034 APPLE KIDS
1035 HEADSTART MONTESSORI HOUSE OF CHILDREN
1036 PODAR HAPPY KIDS
1037 OI PLAYSCHOOL N.R.COLONY
1038 MUD PIE INTERNATIONAL
1039 KIDZEE
1040 THE LITTLE BUDS
1041 LITTLE MILLENNIUM
1042 SHANTI JUNIORS
1043 JOLLY GO KIDZ
1044 KEPHEE PRE SCHOOL
1045 BRIGHT START PLAYSCHOOL
1046 KINDLE KARE
1047 LITTLE ELLY PRESCHOOL - HORAMAVU
1048 BILLY BEARS
1049 KIDS HOME
1050 UNIQID
1051 KADBURY KIDS
1052 THE CIRCLE PRE SCHOOL
1053 NOTTY FEET
1054 KIDS ACADEMY FOR LEARNING
1055 VIVERO INTERNATIONAL PRE-SCHOOL
1056 GOKUL PRE SCHOOL
1057 BACHPAN PLAY SCHOOL
1058 HELLO KIDS
1059 GNAN SRISHTI SCHOOL OF EXCELLENCE
1060 KANGAROO KIDS CLUB AND PRESCHOOL
1061 LITTLE ELLY PRESCHOOL
1062 CASA DEI BAMBINO MONTESSORI
1063 FLORETZ MONTESSORI SCHOOL
1064 LITTLE ELLY PRESCHOOL
1065 BACHPAN PLAY SCHOOL
1066 THE TREE HOUSE PRESCHOOL
1067 BACHPAN PLAY SCHOOL
1068 KARNATAKA PUBLIC SCHOOL
10

1352 EURO KIDS
1353 CORNER STONE MONTESSORI
1354 RENGA MONTESSORI
1355 ATHENA PUBLIC SCHOOL
1356 SAMVIT GURUKUL
1357 LITTLE ELLY PRESCHOOL
1358 BACHPAN PLAY SCHOOL
1359 KUDDLE KIDZ
1360 JACKFRUIT HOUSE
1361 APPLE KIDS
1362 HELLO KIDS
1363 LITTLE ELLY PRESCHOOL
1364 OXFORD HIGHSCHOOL
1365 I PLAY I LEARN
1366 IMPERIAL MOTHERCARE
1367 GOLDEN NURSERY SCHOOL
1368 KIDS R KIDZ
1369 B G NATIONAL PUBLIC HIGH SCHOOL
1370 SIDDAGANGA PUBLIC SCHOOL
1371 SADSHIVANAGAR ENGLISH NURSERY SCHOOL
1372 KIDS WORLD PRE-NURSERY SCHOOL
1373 BAPUJI MEMORIAL SCHOOLS
1374 KIDZEE
1375 PRERANA MONTESSORI HOUSE OF CHILDREN
1376 TIME KIDS
1377 BACHPAN PLAY SCHOOL
1378 BRIGHT KID MONTESSORI HOUSE
1379 HELLO KIDS
1380 TOTS HOUSE PLAY HOUSE & DAY CARE CENTRE
1381 ACTIVE KIDS MONTESSORI
1382 VIHA PLAYSCHOOL AND DAY CARE
1383 SILOAM SCHOLARS SCHOOL
1384 GEETHANJALI MONTESSORI
1385 USHA MONTESSORI SCHOOL
1386 LITTLE ELLY PRESCHOOL
1387 EURO KIDS
1388 KIDZEE
1389 BRIGHT KID MONTESSORI HOUSE
1390 KHALASA PUBLIC SCHOOL
1391 C

1670 TIME KIDS
1671 KIDZEE
1672 LITTLE HEAVEN
1673 DAFFODILS PRE SCHOOL
1674 LITTLE MILLENNIUM
1675 BACHPAN PLAY SCHOOL
1676 SILICON VALLEY HIGH SCHOOL
1677 ZOOL KIDS
1678 CHARLESWORTH NURSERY
1679 KINDER WORLD
1680 CUTE KIDS PLAY SCHOOL & DAY CARE
1681 ST JUDES PLAY SCHOOL AND NURSERY
1682 OMKAR SHISHU VIHAR
1683 ROMASHA VIDYANIKETAN
1684 NEW ARAVIND MEMORIAL SCHOOL
1685 KARA DAY CARE CENTRE
1686 NEEV
1687 SHESHIRA PLAY HOME
1688 DECCAN INTERNATIONAL SCHOOL
1689 TINY STEPS
1690 SPROUT MONTESSORI HOUSE OF CHILDREN
1691 APPLE KIDS
1692 HELLO KIDS
1693 ANANDA CHILDREN PLAY HOUSE
1694 KIDZEE
1695 LITTLE STAR SCHOOL
1696 KREEDARANGA PLAY SCHOOL
1697 BAL BRAHMAS
1698 SHANTI JUNIORS
1699 LITTLE ELLY PRESCHOOL
1700 PEBBLES FUNSKOOL
1701 ABC MONTESSORI
1702 BLOSSOMS INTERNATIONAL PLAYSCHOOL & DAY CARE
1703 KIDZEE
1704 MATRUKRUPA DAY CARE CENTRE & PLAY HOME
1705 TIERRA PRESCHOOL
1706 GURU DARSHAN PUBLIC SCHOOL
1707 HELLO KIDS
1708 FUTURE SCHOLARS PRESCHOOL
1709 A B C ACADEMY
1710 KIDZEE
1711 KI

1983 EURO KIDS
1984 SHLOKA MONTESSORI HOUSE OF CHILDREN
1985 SUNBEAM MONTESSORI
1986 EURO KIDS
1987 BUDDHI SCHOOL
1988 VIDYA VIKAS SCHOOL
1989 MADHUR DAY CARE & PLAY SCHOOL
1990 KIDZ HUT
1991 APPLE KIDS INTERNATIONAL PRE-SCHOOL
1992 KENNETH GEORGE ENGLISH SCHOOL
1993 HELLO KIDS DISCOVERY
1994 BEAR HUG GLOBAL PLAY SCHOOL
1995 HIMALAYA SCHOOL
1996 LETS START
1997 YASHODA LAALAN PAALAN PRE SCHOOL
1998 KIDZEE
1999 LE DAUPHINZ PRESCHOOL
2000 LITTLE MILLENNIUM
2001 SUMMER HILL PLAY HOME
2002 LITTLE MILLENNIUM
2003 TODDLERS KINGDOM
2004 VIDYA VAIBHAV
2005 NIGLI'S PRE SCHOOL
2006 BRILLIANT HIGH SCHOOL
2007 RISE AND SHINE MONTESSORI SCHOOL
2008 BM ENGLISH SCHOOL
2009 SRI CHAITANYA TECHNO SCHOOL
2010 LITTLE MILLENNIUM
2011 HELLO KIDS
2012 KARA 4 KIDS
2013 LITTLE ELLY PRESCHOOL
2014 LE DAUPHINZ PRESCHOOL
2015 INFANT INDIAN SCHOOL
2016 MARIA MONTESSORI SCHOOL
2017 THE FREETHINKING SCHOOL
2018 YOUR KIDS R OUR KIDS
2019 ACHIEVERS ACADEMY
2020 RENNYS PRE SCHOOL & DAY CARE
2021 S S B INTERNATIONAL MON

2296 I PLAY I LEARN
2297 I WONDER Y PRESCHOOL AND DAYCARE
2298 KIDS PARADISE
2299 KIDZ GURUKUL PRESCHOOL
2300 LITTLE HORIZONS MONTESSORI PRESCHOOL
2301 LITTLE RAIDERS
2302 NAKSHATRA MONTESSORI
2303 OI PLAYSCHOOL
2304 RAVINDRA BHARATHI GLOBAL SCHOOL
2305 SPARKLERS PLAY SCHOOL AND ACTIVITY CENTRE
2306 TRIO TOTS PRESCHOOL
2307 HERITAGE KIDS
2308 EKAM SHISHUMANDIRAM
2309 AKKSHAYA KIDS PRE SCHOOL
2313 FIRST STEPS
2314 HAPPYCARE PRESHCOOL AND DAYCARE
2315 KIDS CASTLE PRESCHOOL
2316 LITTLE ELLY PRESCHOOL
2327 KIDZEE
2328 LITTLE ELLY PRESCHOOL
2336 INCARNATIONS MONTESSORI
2337 KIDZEE
2338 KIDZEE LITTLE HEARTS
2339 LITTLE ELLY PRESCHOOL
2340 MUNCHKINS MONTESSORI AND DAY CARE
2341 RAOS MONTESSORI
2342 THE LEARNING CURVE
2343 EDUCHAMPS
2344 KIDZEE
2349 BACHPAN A PLAY SCHOOL
2350 FOOTPRINTS PRE-SCHOOL & DAY CARE CRECHE
2351 GIGGLEZZ PRE SCHOOL
2352 GOODSTART PRESCHOOL AND DAY CARE
2353 I PLAY I LEARN
2354 IBLOOM PRESCHOOL
2355 KIDS CUBE INTERNATIONAL PRE SCHOOL
2357 LITTLE GEMS PRESCHOOL
2358 LITT

2642 LITTLE EINSTEINS PRESCHOOL
2643 BEEHIVE
2644 DISCOVERY MONTESSORI USA
2645 EURO KIDS
2646 HELLO KIDS
2647 OI PLAY SCHOOL
2648 PODAR JUMBO KIDS PLUS
2649 SHANTI JUNIORS
2650 VACA INTERNATIONAL PRE SCHOOL
2651 BAMBY KIDZ
2652 BLOOMING KIDS
2653 BLOSSOMS INTERNATIONAL PLAY SCHOOL
2654 I PLAY I LEARN
2655 I PLAY I LEARN - VIJAY NAGAR
2656 KINDER WORLD
2657 LITTLE KIDZ
2658 PLANET KIDS
2659 SANKALPA PRE SCHOOL & MONTESSORI HOUSE OF CHILDREN
2660 TODDLERS HAVEN
2661 VIDYARAMBH
2662 KIDS NEST
2663 KINDERCARE
2664 EURO KIDS
2665 BACHPAN PLAY SCHOOL
2666 BASIL WOODS PRESCHOOL ACADEMY
2667 CDC KIDS
2668 CHRYSALIS KIDS
2669 EKAM SHISHUMANDIRAM
2670 EURO KIDS
2671 FOOTPRINTS PRE-SCHOOL & DAY CARE
2672 FUTURE MINDS
2673 HELLO KIDS
2674 IKON KIDS
2675 KIDZ PATHSHALA
2676 KIDZEE ITPL
2677 KLAY PREP SCHOOL & DAYCARE
2678 KREYO AT THE COURTYARD
2679 LA CHILD CARE
2680 LITTLE EDEN PRESCHOOL AND DAYCARE CENTRE
2681 LITTLE ELLY PRESCHOOL
2682 LITTLE MILLENNIUM
2683 LITTLEMILLENNIUM
2684 MIRACLE KIDS 

2925 B G NATIONAL PUBLIC HIGH SCHOOL
2926 CAPITAL PUBLIC SCHOOL
2927 OXFORD HIGHSCHOOL
2928 GOPALAN NATIONAL SCHOOL
2929 BANGALORE PUBLIC SCHOOL
2930 DEVA MATHA CENTRAL SCHOOL
2931 A.S. KUPPARAJU & BROTHERS VIDYANIKETAN
2932 VIDYA NIKETAN HIGH SCHOOL
2933 BAPUJI MEMORIAL SCHOOLS
2934 SILICON VALLEY HIGH SCHOOL
2935 EURO SCHOOL
2936 WISDOM INTERNATIONAL SCHOOL
2937 INDUS INTERNATIONAL SCHOOL
2938 SRI CHAITANYA TECHNO SCHOOL
2939 A V EDUCATION SOCIETY
2940 SEEMA ENGLISH SCHOOL
2941 BANGALORE SCOTTISH SCHOOL
2942 B M VALLEY SCHOOL
2943 SUDHARSHAN VIDYA MANDIR
2944 FRANK PUBLIC SCHOOL
2945 ACTS SECONDARY SCHOOL
2946 ST JOHNS HIGH SCHOOL
2947 GREEN VALLEY ENGLISH SCHOOL
2948 DECCAN INTERNATIONAL SCHOOL
2949 FREEDOM INTERNATIONAL SCHOOL
2950 ST.MERIES HIGH SCHOOL
2951 TINY BUDS CRECHE
2952 SANS PUBLIC SCHOOL
2953 G. K NAIDU SCHOOL
2954 FEDERAL PUBLIC SCHOOL
2955 HERITAGE PUBLIC SCHOOL
2956 NARAYANA E-TECHNO SCHOOL
2957 PRAKRIYA GREEN WISDOM SCHOOL
2958 RES CONVENT
2959 SRI CHAITANYA TECHNO S

In [18]:
Details_df.shape

(3036, 7)

In [489]:
Details_df.to_csv(r"/home/sumit/Scrape_Project/School_Details Batch-2 02022020.csv")

In [ ]:
24-320
990-1020

In [422]:
def scraping_links():
    driver= webdriver.Chrome()##Defining the driver.
    driver.implicitly_wait(10)## Implicit wait to load browser.
#     Details_df=pd.DataFrame(columns=["Name_Institution","Contact_Person","Contact_No","Email","Location","Website","Address"])
    for x in range(0,len(links_df)):
        if x%200==0:
            driver.delete_all_cookies()
        try:
            driver.get(links_df[x])
            WebDriverWait(driver,10).until(cond.presence_of_all_elements_located((By.CLASS_NAME,'r_source-review')))## Implicit wait to load webpage.
        except TimeoutException:
            driver.get(links_df[x])
            WebDriverWait(driver,10).until(cond.presence_of_all_elements_located((By.CLASS_NAME,'r_source-review')))## Implicit wait to load webpage.
        except WebDriverException:
            driver= webdriver.Chrome()##Defining the driver.
            driver.implicitly_wait(10)
            driver.get(links_df[x])
            WebDriverWait(driver,10).until(cond.presence_of_all_elements_located((By.CLASS_NAME,'r_source-review')))## Implicit wait to load webpage.
            #         time.sleep(10)## 10 Seconds wait for all elements to be loaded.
        details_dict={}## Dictionary for each detail to append into dataframe.
        col_name=list(Details_df.columns)
        ## Name of Institution
        details_dict[col_name[0]]=driver.find_element_by_xpath("/html/body/div[7]/div[2]/div[5]/div[1]/h1").text
        ## Location
        details_dict[col_name[4]]=driver.find_element_by_xpath("/html/body/div[7]/div[2]/div[7]/div[3]/div[3]/div[2]").text
        ## Address
        add=driver.find_element_by_id('contact').text
#         print(add)
        details_dict[col_name[6]]=add.replace("\n",",")
#         ",".join(add.split("\n")[1:-3])
        ## Contact Details block from website.
        contact=driver.find_elements_by_class_name('r_source-review')[-2]
        if len(contact.text.split("\n"))>1:
            if not re.match(r"\d+",contact.text.split("\n")[1]):
                ##Contact Person
                details_dict[col_name[1]]=contact.text.split("\n")[1]
#             print(details_dict[col_name[1]])
            else: details_dict[col_name[1]]=None        
        
            mob,mail,web=find_details(contact.text.split("\n"))
            ## Phone No
            details_dict[col_name[2]]=mob
            ## Mail
            details_dict[col_name[3]]=mail
            ## Website
            details_dict[col_name[5]]=web
        else:
             ## Phone No
            details_dict[col_name[2]]=None
            ## Mail
            details_dict[col_name[3]]=None
            ## Website
            details_dict[col_name[5]]=None
#         phno=contact_p.text.split("\n")[2]
#         if len(contact_p.text.split("\n"))>3:
#             if re.match(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",contact_p.text.split("\n")[3]):
#                 email=contact_p.text.split("\n")[3]
#             else:
#                 phno=list(phno).append(contact_p.text.split("\n")[3])
#         if len(contact_p.text.split("\n"))>3:
#         websit=contact_p.text.split("\n")[3]
        Details_df=Details_df.append(details_dict,ignore_index=True)
        print(x,details_dict[col_name[0]])
#         break
    driver.close()
    return Details_df

In [423]:
scraping_links()

UnboundLocalError: local variable 'Details_df' referenced before assignment

In [407]:
dataset2

,Name_Institution,Contact_Person,Contact_No,Email,Location,Website,Address
0,LITTLE PEARLS PLAYSCHOOL,Ms. Sonia Nanda,9891247399,littlepearls.schoolnoida@gmail.com,SECTOR 29 NOIDA,www.littlepearlsplayschoolnoida.com,"Venue,Plot no. F1 & F3, Sector 29 Brahmaputra ..."
1,DECORUM CASTLE PLAY NURSERY SCHOOL,Ms. Surbhi,022305899/ 09899839155,None,SHAHDARA,None,"Venue,452/2C Bhola Nath Nagar, Shahdra ,,Near ..."
2,EURO KIDS,None,8586092984/ 8800553780,eurokidsnajafgarh@gmail.com,NAJAFGARH,None,"Venue,A 8 & 9 Anand Vihar, Nangli Sakrawati Ne..."
3,MOTHER'S PRIDE,Ms aditi singh,+911244045869,aditi.singh@mothersprideonline.com,SECTOR 51 GURGAON,www.motherspridepreschool.com,"Venue,N 55, May feild Garden, Sec 51 Gurgoan, ..."
4,ANKITAS MOTHERS PRIDE,Mrs. Manjula Singh,,None,SHIPRA SUNCITY,None,"Venue,Phase II,,Shipra Suncity,Shipra Suncity,..."
5,AADHARSHILA YOUR KIDS PARADISE,Ms Sadaf,+911244360582,aadharshila_kids@yahoo.com,SOUTH CITY II,www.aadharshilapreschool.org,"Venue,Q1 block, Behind B Block Market,South Ci..."
6,THE FLOWERING TREE PRE SCHOOL,Mrs. Pramila Sharma,,None,DLF PHASE I,http://www.thefloweringtree.in,"Venue,Silver Oak Aprt,,1101, Lane H-35,DLF Pha..."
7,TRINITY TINY TOTS PLAY SCHOOL,Mrs. Malhotra,+911204262472,rakeshtrinity@gmail.com,SECTOR 62 NOIDA,None,"Venue,C-30/1 Opp Indian Oil Apartment,Sector 6..."
8,GD GOENKA LA PETITE,+919999908686,+919999908686,None,PASCHIM VIHAR,www.gdglp.com,"Venue,BG / 7,,Paschim Vihar Near Jwala Heri Ma..."
9,JOY RIDE SCHOOL FOR JUNIORS,Ms. Anupama Bhutani,+911292223232,None,SECTOR 15 FARIDABAD,http://www.joyrideschool.com,"Venue,Sector - 15,,Faridabad Behind APJ School..."


In [324]:
dataset.to_csv(r"/home/sumit/Scrape_Project/Details_df[2000] 01022020.csv",index=False)